In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
fname = "Protein_alignment.txt"
fname2 = "BLOSUM62.txt"
ar2 = np.loadtxt(fname, delimiter="\t", dtype='str')
ar3 = np.loadtxt(fname2, dtype='str')

In [3]:
def print_matrix(mat):
    for i in range(len(mat)):
        print(mat[i])

In [4]:
def convert_to_seq(ar):
    dna = []
    number_of_items = 0
    for i in range(ar.shape[0]):
        if ar[i][0] != ">":
            dna.append([ar[i][j] for j in range(len(ar[i]))])
        else:
            number_of_items += 1
    return np.array(dna), number_of_items

In [5]:
 def form_complete_seq(dna, n):
    final = []
    start = 0
    inc = int(len(dna) / n)
    end = inc

    while end <= dna.shape[0]:
        temp = []
        for i in range(start, end):
            for j in range(len(dna[i])):
                temp.append(dna[i][j])
        temp = np.array(temp)
        final.append(temp)
        start = start + inc
        end = end + inc

    return np.array(final)

In [6]:
def find_score(a, b, dmat):
    x = 0
    y = 0
    if a == "-":
        a = "*"
    if b == "-":
        b = "*"
    for i in range(dmat.shape[0]):
        if a == dmat[0][i]:
            x = i
        if b == dmat[0][i]:
            y = i
    return dmat[x][y]

In [7]:
def find_dist(m, n, final, dmat):
    # here m, n = seq1, seq2
    count = 0
    t = min(len(final[m]), len(final[n]))
    left = max(len(final[m]), len(final[n])) - t

    for i in range(t):
        temp = find_score(final[m][i], final[n][i], dmat)
        count += int(temp)
    count += (left*(-4))
    
    return count

In [8]:
 def generate_dist_matrix(final, k, dmat):
    mat = [[0 for i in range(k)] for j in range(k)]

    # here x, y = seq1, seq2
    for x in range(k):
        for y in range(k):
            mat[x][y] = find_dist(x, y, final, dmat)
            
    return np.array(mat)

In [9]:
dmat, m = convert_to_seq(ar3)

p1, n = convert_to_seq(ar2)
p2 = form_complete_seq(p1, n)
dist_matrix = generate_dist_matrix(p2, n, dmat)

print_matrix(dist_matrix)

<ipython-input-4-7e6695ddb7d4>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(dna), number_of_items


[589 168 255 221 201 200 194 212 210 214]
[168 591 237 332 314 270 332 347 359 356]
[255 237 579 331 346 333 302 331 330 326]
[221 332 331 596 459 425 444 468 485 482]
[201 314 346 459 583 499 423 480 487 484]
[200 270 333 425 499 575 388 451 442 438]
[194 332 302 444 423 388 575 455 481 478]
[212 347 331 468 480 451 455 587 522 518]
[210 359 330 485 487 442 481 522 591 584]
[214 356 326 482 484 438 478 518 584 591]


In [18]:
res = {idx + 1 : dist_matrix[idx] for idx in range(len(dist_matrix))} 

In [11]:
# dist_matrix.tofile('Ndistance.txt',sep=',')

In [20]:
pd.DataFrame(res).to_csv("Pdistance.txt")